In [1]:
import numpy as np
import scipy.stats as stats
import pymc3 as pm
from theano import shared
import matplotlib.pyplot as plt
import theano
import theano.tensor as tt
floatX = "float32"

In [2]:
%load_ext watermark
%watermark -v -m -p numpy,sklearn,pandas,matplotlib,seaborn,pymc3,theano

CPython 2.7.13
IPython 5.3.0

numpy 1.13.0
sklearn 0.18.1
pandas 0.19.2
matplotlib 2.0.0
seaborn 0.7.1
pymc3 3.3rc1
theano 1.0.0

compiler   : GCC 4.2.1 Compatible Apple LLVM 8.0.0 (clang-800.0.42.1)
system     : Darwin
release    : 17.3.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [3]:
from sklearn.datasets import make_blobs

n_samples = 100
nk = 5
random_state = 170
X, y = make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5, 1.5, .8], random_state=random_state, centers=5)
X = X.astype('float32')
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.viridis);

In [4]:
K = 15
def stick_breaking(beta):
    portion_remaining = tt.concatenate([[1], tt.extra_ops.cumprod(1 - beta)[:-1]])
    return beta * portion_remaining

# Log likelihood of Gaussian mixture distribution
from pymc3.math import logsumexp
solver = tt.slinalg.Solve(A_structure="lower_triangular", lower=True)
def GMM_logp(weight, mus, chol):
    def logp_(value):
        Ncomp = len(chol)
        logps = []
        for i in range(Ncomp):
            mu = mus[:, i]
            chol_cov = chol[i]
            k = chol_cov.shape[0]

            delta = value.reshape((-1, k)) - mu
            delta_trans = solver(chol_cov, delta.T)

            result = k * tt.log(2 * np.pi)
            result += 2.0 * tt.sum(tt.log(tt.nlinalg.diag(chol_cov)))
            result += (delta_trans ** 2).sum(axis=0).T
            logps.append(tt.log(weight[i]) + -0.5*result)
        return tt.sum(logsumexp(tt.stacklists(logps), axis=0))
    return logp_

In [5]:
with pm.Model() as model:
    alpha = pm.Gamma('alpha', 1., 1.)
    beta = pm.Beta('beta', 1., alpha, shape=K)
    w = pm.Deterministic('w', stick_breaking(beta))
                         
    sd_dist = []
    packed_chol = []
    chol = []
    for i in range(K):
        sd_dist.append(pm.HalfCauchy.dist(beta=2.5))
        packed_chol.append(pm.LKJCholeskyCov('chol_cov_%i'%i, eta=2, n=2, sd_dist=sd_dist[i]))
        chol.append(pm.expand_packed_triangular(2, packed_chol[i], lower=True))
    
    mus = pm.Normal('mu', 0, sd=10, shape=(2, K))

    obs = pm.DensityDist('obs', GMM_logp(w, mus, chol), observed=X)

In [8]:
obs.distribution.logp(theano.shared(X[:10])).ndim

0